In [28]:


import time

import json
import re

import pandas as pd
import os
import spacy
from spacy import displacy
from textblob import TextBlob, Word
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

from tqdm import tqdm
from english_words import english_words_set



[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Francesco\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Francesco\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Francesco\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
# Calculate scores

In [3]:
files=[x for x in os.listdir('NEW_csv_folders/') if x.endswith('.csv')]
df_dicts={x:pd.read_csv('NEW_csv_folders/'+x) for x in os.listdir('NEW_csv_folders/') if x.endswith('.csv') }

In [4]:
df_dicts

{'CONCAT_Orthodox Jewish Version.csv':                      Edition        Book  Chapter  \
 0      Orthodox Jewish Bible     Genesis        1   
 1      Orthodox Jewish Bible     Genesis        1   
 2      Orthodox Jewish Bible     Genesis        1   
 3      Orthodox Jewish Bible     Genesis        1   
 4      Orthodox Jewish Bible     Genesis        1   
 ...                      ...         ...      ...   
 30560  Orthodox Jewish Bible  Revelation       22   
 30561  Orthodox Jewish Bible  Revelation       22   
 30562  Orthodox Jewish Bible  Revelation       22   
 30563  Orthodox Jewish Bible  Revelation       22   
 30564  Orthodox Jewish Bible  Revelation       22   
 
                                                    Verse  \
 0      1  In the beginning Elohim bara hashomayim (th...   
 1      2  And the earth was tohu vavohu (without form...   
 2      3  And Elohim said, Let there be light: and th...   
 3      4  And Elohim saw the light, that it was tov (...   
 4     

In [29]:
#temporal file
files=["CONCAT_Orthodox Jewish Bible.csv"]
df_dicts={files[0]:pd.read_csv('NEW_csv_folders/'+files[0])}

'god' in en_words

In [30]:
nlp = spacy.load("en_core_web_sm")

for file in files:
    proper_nouns_=[]
    sentiment_=[]
    subjectivity_=[]
    summary_=[]
    for j in tqdm(range(len(df_dicts[file]))):
        #print(j)
        verse=df_dicts[file]['Verse'][j]
        doc=nlp(verse)
        #propernouns
        entities_=[str(token) for token in doc if token.pos_=='PROPN']
        #sentiment/subjectivity
        blob= TextBlob(verse)
        verse_polarity=blob.sentiment_assessments.polarity
        verse_subjectivity= blob.sentiment_assessments.subjectivity
        verse_assessment= blob.sentiment_assessments.assessments
        verse_polarity,verse_subjectivity, verse_assessment
        #summary
        summary_nouns=[]
        for word,tag in blob.tags:
            if tag == 'NN':
                summary_nouns.append(word.lemmatize())
        #counters
        counter={x:0 for x in summary_nouns}
        counter_entities={x:0 for x in entities_}
        for word_ in summary_nouns:
            counter[word_]+=1
        for entity_ in entities_:
            counter_entities[entity_]+=1
        ordered_summary=[[k,v] for k,v in counter.items()]
        ordered_summary= [x[0] for x in sorted(ordered_summary,key=lambda x:x[1], reverse=True)]
        ordered_entities=[[k,v] for k,v in counter_entities.items()]
        ordered_entities= [x[0] for x in sorted(ordered_entities,key=lambda x:x[1], reverse=True)]
        #append
        #print(entities_, verse_polarity, verse_subjectivity, ordered_summary)
        proper_nouns_.append(';'.join([str(x) for x in ordered_entities]))
        sentiment_.append(verse_polarity)
        subjectivity_.append(verse_subjectivity)
        summary_.append(';'.join(ordered_summary))
    df_dicts[file]['Proper nouns']=proper_nouns_
    df_dicts[file]['Sentiment']=sentiment_
    df_dicts[file]['Subjectivity']=subjectivity_
    df_dicts[file]['Summary']=summary_
    df_dicts[file].to_csv('NEW_csv_folders/'+file, index=False,header=True)
    print(file + ' saved')
    

100%|████████████████████████████████████████████████████████████████████████████| 31024/31024 [08:02<00:00, 64.35it/s]


CONCAT_Orthodox Jewish Bible.csv saved


In [156]:
';'.join(ordered_summary)

'[;kindness'

# clean proper nouns and summaries

In [31]:
file_name="NEW_csv_folders/CONCAT_Orthodox Jewish Bible.csv"
df_new=pd.read_csv(file_name)

'spirit' in en_words #even if it is personifiable, i'll exclude it 

In [32]:
df_new[1200:1300]

,Edition,Book,Chapter,Verse,Proper nouns,Sentiment,Subjectivity,Summary
1200,Orthodox Jewish Bible,Genesis,41,5 And he slept and dreamed a chalom the second...,hinei;sheva;beri'ot,0.000000,0.000000,chalom;time;hinei;grain;stalk;beri'ot;tovot
1201,Orthodox Jewish Bible,Genesis,41,"6 And, hinei, seven thin heads of grain scorch...",hinei;east;tzomechot,-0.400000,0.850000,hinei;grain;wind;tzomechot
1202,Orthodox Jewish Bible,Genesis,41,7 And the seven thin heads of grain devoured t...,mele'ot;Pharaoh;hinei,0.018750,0.700000,grain;fat;mele'ot;chalom
1203,Orthodox Jewish Bible,Genesis,41,8 And it came to pass in the boker that his ru...,Pharaoh;Mitzrayim;Egypt,-0.500000,1.000000,boker;ruach;chartummei;chachamim;thereof;chalo...
1204,Orthodox Jewish Bible,Genesis,41,"9 Then the sar hamashkim spoke unto Pharaoh, s...",sar;Pharaoh,0.000000,0.000000,sar;hamashkim;day
...,...,...,...,...,...,...,...,...
1295,Orthodox Jewish Bible,Genesis,43,"5 But if thou wilt not send him, we will not g...",Ye;achichem,-0.155556,0.288889,wilt;ish;face
1296,Orthodox Jewish Bible,Genesis,43,"6 And Yisroel said, Why dealt ye so ill with m...",Yisroel,-0.500000,1.000000,ye;dealt;ach;brother
1297,Orthodox Jewish Bible,Genesis,43,"7 And they said, The ish asked in detail about...",kindred;Avichem,0.052910,0.420106,detail;moledet;ach;inquiry;]
1298,Orthodox Jewish Bible,Genesis,43,"8 And Yehudah said unto Yisroel Aviv, Send na'...",Yehudah;Yisroel;Aviv,-0.025568,0.500000,na'ar;thou


In [33]:
en_words=[x for x in english_words_set]
exception_words=['god']

In [34]:
Word('women').lemmatize()

'woman'

In [35]:
def clean_proper_nouns(df_new):
    for i,row in tqdm(enumerate(df_new['Proper nouns'])):
        try:
            name_set=df_new['Proper nouns'][i].split(';')
        except:
            name_set=[]
        right_names=[]
        checks=[]
        #print(name_set)
        #if the words in dict is in the bible word (kind of lemma)
        for j,name_ in enumerate(name_set):
            w=Word(name_.lower())
            name=w.lemmatize()
            name_check=[name==x for x in en_words ]
            if any(name_check) and name not in exception_words:
                pass
            else:
                right_names.append(name_) #not lemmatized
            #print(i,j, name_,any(name_check))
        right_names=[x for x in right_names if x.strip().istitle()]
        #print(right_names)

        df_new.iloc[i,4]=';'.join(right_names)

def clean_summary(df_):
    for i,summary in enumerate(df_['Summary']):
        if str(summary):
            df_.iloc[i,7]=';'.join([x for x in str(summary).split(';') if x.isalpha()])
        

    
        
   


In [36]:
clean_proper_nouns(df_new)
clean_summary(df_new)  

31024it [05:13, 99.03it/s] 


In [37]:
df_new

,Edition,Book,Chapter,Verse,Proper nouns,Sentiment,Subjectivity,Summary
0,Orthodox Jewish Bible,Genesis,1,1 In the beginning Elohim bara hashomayim (the...,Elohim,0.000,0.000000,beginning;bara;hashomayim;haaretz;earth
1,Orthodox Jewish Bible,Genesis,1,"2 And the earth was tohu vavohu (without form,...",Ruach;Elohim,0.000,0.400000,face;earth;vavohu;form;void;darkness;deep
2,Orthodox Jewish Bible,Genesis,1,"3 And Elohim said, Let there be light: and the...",Elohim,0.400,0.700000,light
3,Orthodox Jewish Bible,Genesis,1,"4 And Elohim saw the light, that it was tov (g...",Elohim,0.500,0.666667,light;ohr;choshech;darkness
4,Orthodox Jewish Bible,Genesis,1,"5 And Elohim called the light Yom (Day), and t...",Yom;Elohim;Lailah;Echad,0.325,0.516667,light;darkness;erev;boker;morning
...,...,...,...,...,...,...,...,...
31019,Orthodox Jewish Bible,Revelation,22,"17 And the Ruach Hakodesh and the Kallah say, ...",Bo;Ruach;Hakodesh;Kallah;Mayim;Chayyim,0.400,0.800000,hearing;let;thirsting
31020,Orthodox Jewish Bible,Revelation,22,18 I give solemn edut to everyone hearing the ...,Hashem,0.000,0.000000,sefer;edut;everyone;divrei;anyone;makkot
31021,Orthodox Jewish Bible,Revelation,22,19 And if anyone takes away from the devarim o...,Hashem;Etz;Ir,0.000,0.000000,sefer;anyone;devarim;prophecy;share
31022,Orthodox Jewish Bible,Revelation,22,20 He who gives solemn edut to these things sa...,Omein;Adoneinu;Yehoshua,0.000,0.000000,edut;bahlt


df_new['Edition']=["Young's Literal Translation" for x in df_new['Edition']]
df_new

In [38]:
df_new.to_csv(file_name,index=False,header=True)

In [39]:
df_random=pd.read_csv('NEW_csv_folders/CONCAT_Orthodox Jewish Bible.csv')
df_random

,Edition,Book,Chapter,Verse,Proper nouns,Sentiment,Subjectivity,Summary
0,Orthodox Jewish Bible,Genesis,1,1 In the beginning Elohim bara hashomayim (the...,Elohim,0.000,0.000000,beginning;bara;hashomayim;haaretz;earth
1,Orthodox Jewish Bible,Genesis,1,"2 And the earth was tohu vavohu (without form,...",Ruach;Elohim,0.000,0.400000,face;earth;vavohu;form;void;darkness;deep
2,Orthodox Jewish Bible,Genesis,1,"3 And Elohim said, Let there be light: and the...",Elohim,0.400,0.700000,light
3,Orthodox Jewish Bible,Genesis,1,"4 And Elohim saw the light, that it was tov (g...",Elohim,0.500,0.666667,light;ohr;choshech;darkness
4,Orthodox Jewish Bible,Genesis,1,"5 And Elohim called the light Yom (Day), and t...",Yom;Elohim;Lailah;Echad,0.325,0.516667,light;darkness;erev;boker;morning
...,...,...,...,...,...,...,...,...
31019,Orthodox Jewish Bible,Revelation,22,"17 And the Ruach Hakodesh and the Kallah say, ...",Bo;Ruach;Hakodesh;Kallah;Mayim;Chayyim,0.400,0.800000,hearing;let;thirsting
31020,Orthodox Jewish Bible,Revelation,22,18 I give solemn edut to everyone hearing the ...,Hashem,0.000,0.000000,sefer;edut;everyone;divrei;anyone;makkot
31021,Orthodox Jewish Bible,Revelation,22,19 And if anyone takes away from the devarim o...,Hashem;Etz;Ir,0.000,0.000000,sefer;anyone;devarim;prophecy;share
31022,Orthodox Jewish Bible,Revelation,22,20 He who gives solemn edut to these things sa...,Omein;Adoneinu;Yehoshua,0.000,0.000000,edut;bahlt


In [40]:
df_random[df_random['Book']=='Ruth']

,Edition,Book,Chapter,Verse,Proper nouns,Sentiment,Subjectivity,Summary
19474,Orthodox Jewish Bible,Ruth,1,1 Now it came to pass in the yamim when HaShof...,Beit;Lechem;Yehudah;Moav,0.214286,0.571429,yamim;famine;land;man;sadei;isha;wife;banim
19475,Orthodox Jewish Bible,Ruth,1,2 And the shem (name) of the ish (man) was Eli...,Elimelech;Naomi;Machlon;Kilyon;Ephrathites;Bei...,0.000000,0.000000,shem;name;man;isha;wife;banim;sadei
19476,Orthodox Jewish Bible,Ruth,1,"3 And Elimelech, Naomiʼs husband, died; and sh...",Elimelech,0.050000,0.200000,husband;banim
19477,Orthodox Jewish Bible,Ruth,1,4 And they took them wives of the nashim (wome...,Moav;Orpah;Ruth,-0.062500,0.187500,shem;name;nashim;one;eser;shanim
19478,Orthodox Jewish Bible,Ruth,1,5 And Machlon and Kilyon died also both of the...,Machlon;Kilyon,0.050000,0.200000,isha;yeladim;ish
...,...,...,...,...,...,...,...,...
19554,Orthodox Jewish Bible,Ruth,4,18 Now these are the toldot of Peretz; Peretz ...,Peretz;Chetzron,0.000000,0.000000,toldot
19555,Orthodox Jewish Bible,Ruth,4,"19 And Chetzron fathered Ram, and Ram fathered...",Chetzron;Amminadav,0.000000,0.000000,NaN
19556,Orthodox Jewish Bible,Ruth,4,"20 And Amminadav fathered Nachshon, and Nachsh...",Nachshon;Amminadav,0.000000,0.000000,NaN
19557,Orthodox Jewish Bible,Ruth,4,"21 And Salmon fathered Boaz, and Boaz fathered...",Boaz;Oved,0.000000,0.000000,NaN
